In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import numpy as np
import math
from matplotlib.patches import Rectangle, Circle
from textwrap import wrap

"""
master_input.parquet
master_output.parquet
play_level.parquet
player_analysis.parquet
spatial_features.parquet
supplementary.parquet
trajectories.parquet
"""
base_dir = "../consolidated_data/"

In [9]:
play_keys = ['game_id', 'play_id']
player_keys = play_keys + ['nfl_id']
player_dtls = ['player_name', 'player_height', 'player_weight', 'player_birth_date', 'player_position', 'player_side', 'player_role', 'num_frames_output']
player_mvmt = ['frame_id', 'x', 'y']

# field extents
ymin = 0
ymax = 53.3

xmin = 0
xmax = 120

input_cols = player_keys + player_dtls + player_mvmt + ['player_to_predict', 'play_direction', 'absolute_yardline_number', 's', 'a', 'dir', 'o', 'num_frames_output', 'ball_land_x', 'ball_land_y', 'week']
input_df = pd.read_parquet(f'{base_dir}/master_input.parquet')
supp_df = pd.read_parquet(f'{base_dir}/supplementary.parquet')

output_cols = player_keys + player_mvmt
output_df = pd.read_parquet(f'{base_dir}/master_output.parquet')

supp_input_df = pd.merge(input_df, supp_df, on=['game_id', 'play_id'], how='left')
supp_input_df.loc[:, ["num_frames_input"]] = game_df.groupby(["game_id", "play_id", "nfl_id"])["frame_id"].transform('max')

In [20]:
# 2023090700 1679
game_id = 2023090700
play_id = 1679
game_df = supp_input_df.loc[((supp_input_df['game_id'] == game_id) & (supp_input_df['play_id'] == play_id))]

In [22]:
# players
game_df[['nfl_id', 'player_name']].drop_duplicates()

,nfl_id,player_name
7552,44888,Alex Anzalone
7583,55910,Brian Branch
7614,47888,C.J. Gardner-Johnson
7645,44906,Cameron Sutton
7676,53953,Jerry Jacobs
7707,54562,Kerby Joseph
7738,54653,Malcolm Rodriguez
7769,41325,Jerick McKinnon
7800,46213,Justin Watson
7831,53591,Noah Gray


In [26]:
offense = game_df.loc[game_df['player_side'] == 'Offense']
defense = game_df.loc[game_df['player_side'] == 'Defense']

In [29]:
# for each defender - who can they see in each frame
defense.loc[defense['frame_id'] == 1, ['nfl_id', 'frame_id', 'play_direction', 'o', 'x', 'y']]

,nfl_id,frame_id,play_direction,o,x,y
7552,44888,1,right,261.49,50.85,32.14
7583,55910,1,right,318.47,50.82,20.28
7614,47888,1,right,276.89,56.56,28.26
7645,44906,1,right,293.04,53.48,13.05
7676,53953,1,right,198.19,56.75,42.68
7707,54562,1,right,260.42,56.11,35.60
7738,54653,1,right,288.86,50.72,25.97


In [ ]:
# get approximate field of vision
def getABC(x, y, o):
    
    